In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not edit it
import jinko_helpers as jinko
import sys

sys.path.insert(0, "../lib")

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cookbook specifics imports
import json
import io
import zipfile
import pandas as pd

## Select calibration of interest

In [ ]:
"""
An optional folder location to store debug files
"""

folder = "."
# Set this to True to write debugging files
write_files = True

""" 
Calibration short id can be retrieved from the URL of your calibration in Jinko, pattern is `https://jinko.ai/<calibration_short_id>`
"""
calibration_short_id = "ca-hIAx-A64d"
# Choose a spefic revision. By default we return the last version
revision = 19
# Choose a specific label. By default we return the last version
label = None
response = jinko.get_project_item(
    sid=calibration_short_id, revision=revision, label=label
)
calibration_core_item_id, calibration_snapshot_id = (
    response["coreId"]["id"],
    response["coreId"]["snapshotId"],
)

# # Uncomment the following if you want to use the latest completed or stopped version
# response = jinko.get_latest_calib_with_status(shortId=calibration_short_id, statuses=["completed", "stopped"])
# calibration_core_item_id, calibration_snapshot_id = response["coreItemId"], response["snapshotId"]

print(
    f"Picked Calibration with coreItemId: {calibration_core_item_id}, snapshotId: {calibration_snapshot_id}"
)

status = jinko.make_request(
    f"/core/v2/calibration_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/status",
    method="GET",
).text
print(f"It has status: {status}")

## Debug data tables

In [ ]:
sorted_patients = jinko.make_request(
    path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/sorted_patients",
    method="POST",
    json={"sortBy": "optimizationWeightedScore", "iteration": 10},
).json()
if len(sorted_patients) > 0:
    best_patient = sorted_patients[0]
    print(f"best_patient: {best_patient}")

    response = jinko.make_request(
        path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/augment_data_tables",
        method="POST",
        json={
            "patientId": best_patient["patientNumber"],
            "iteration": best_patient["iteration"],
        },
    )
    archive = zipfile.ZipFile(io.BytesIO(response.content))
    filenames = archive.namelist()
    print(f"CSV files in archive: {filenames}")
    for filename in filenames:
        print(f"Extracting : {filename}")
    csv_content = archive.read(filename).decode("utf-8")
    datatable = pd.read_csv(io.StringIO(csv_content))
    display(datatable)

## Debug scoring or solving errors

In [ ]:
# Choose the iteration number you wish to debug, 1 is a good guess if you do not know where to start
iteration = 1

scoring_errors = jinko.make_request(
    path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/errors",
    method="POST",
    json={"scalarId": "optimizationWeightedScore", "iteration": iteration},
).json()
if len(scoring_errors["errors"]) <= 0:
    print("No error for this iteration")
else:
    first_error = scoring_errors["errors"][0]
    patient_id = first_error["patientId"]
    print(f"Patient {patient_id} has an error:\n{first_error["scoringError"]}")

Based on the above information, you may want to inspect the solver output for a given timeseries ID on a given protocol arm. 
Ideally we should be able to parse the above message and extract the relevant information but for now you will have 
to input these manually yourself.

In the above example, the message is "constituent DefaultComp.CO_2 on arm baz is missing from the scoring evaluation context"
so the protocol arm ID is "baz" and the timeseries ID is "DefaultComp.CO_2"

In [ ]:
# protocol arm ID of interest
arm_id = "baz"
# timeseries ID of interest
timeseries_id = "DefaultComp.CO_2"

model_results = jinko.make_request(
    path=f"/core/v2/result_manager/calibration/{calibration_core_item_id}/snapshots/{calibration_snapshot_id}/timeseries/per_patient",
    method="POST",
    json={"iteration": iteration, "patientId": patient_id, "select": [timeseries_id]},
).json()
model_result = next(
    (item for item in model_results if item["scenarioArm"] == arm_id), None
)
if model_result is None:
    print(f"No ModelResult found for arm {arm_id}")
else:
    print(model_result["error"])